# Data Processing
## Index
1. Figures from multiple data sets
2. Fitting Critical exponents (m, chi, c_v)
3. Fitting Dynamical Critical Exponent
4. Simulation performance


## 1. Figures from multiple data sets

In [ ]:
# Figures for c_v, chi, and m^2 are made from three data sets, L = 10, L = 20, L = 40

import numpy as np
import matplotlib.pyplot as plt

# Specify where to find data sets
data_dir = './exported_data/'

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'triple_2000_' 
fig_name = fig_dir + identifier

# Properties of the data
data_names = ['c_v', 'magnetisation', 'chi']
L_range = [10, 20, 40]
T_data = 81

# Initialization
y_data = np.zeros((len(L_range), T_data))
x_data = np.zeros((len(L_range), T_data))

# Plot markup
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

for data_name in data_names:
    
    # Import the data from the selected data sets 
    for i, L in enumerate(L_range):
        name = 'saved_data_L' + str(L) +'_SW_2000.npz'
        npzfile = np.load(data_dir + name)
        y_data[i] = npzfile[data_name][:,0]
        y_err[i] = npzfile[data_name][:,1]
        x_data[i] = npzfile['temperature'][:,0]
    
    # Plot imported data
    markers = ['+', 'x', '.']
    for k, x in enumerate(x_data):
        # Normal plot or errorbar plot
        plt.plot(x, y_data[k], markers[k], markersize = 6)
        #plt.errorbar(x, y_data[k], yerr = y_err[k], fmt = markers[k], markersize = 6, capsize = 4)
        plt.xlabel('$\mathrm{k_B T/J}$', fontsize=18)
    plt.legend(['L = 10', 'L = 20', 'L = 40'])

    # Plot markup
    if data_name == 'c_v':
        plt.ylabel('$\mathrm{C_v}$', fontsize=18)
    if data_name == 'magnetisation':
        plt.ylabel('$ m^{2} $', fontsize=18)   
    if data_name == 'chi':
        plt.ylabel('$\chi$', fontsize=18)
    
    plt.tight_layout()
    full_dir_name = fig_name + data_name + '.png'
    plt.savefig(fig_name + data_name + '.png')
    plt.close()
print('Figures are saved to:', fig_dir)

## 2. Fitting Critical exponents (m, chi, c_v)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from data_processing import fit_function
#%matplotlib notebook

# Initialisation

## Speficy where to load data
data_dir = './exported_data/saved_data_'
data_set = 'L40_SW_5000'
data_extension = '.npz'
data_location = data_dir + data_set + data_extension

## Specify where to store data
fig_dir = './exported_figs/'
quantity = 'chi'
identifier = '_fitted'
fig_extension = '.png'
fig_name = fig_dir + data_set + '_' + quantity + identifier

## Data specifications
boundaries_neg = np.array([-0.301, -0.074]) # Fitting range for (T-T_c)<0
boundaries_pos = np.array([0.074, 0.29]) # Fitting range for (T-T_c)>0

## Want log or normal scale
LOG = True

## Want to show figure or save figure
show = True

# Calculation & plot making
popt_neg, fit_err_neg, xdata_fit_neg, ydata_fit_neg = fit_function(data_location, quantity, boundaries_neg, False, False)
popt_pos, fit_err_pos, xdata_fit_pos, ydata_fit_pos = fit_function(data_location, quantity, boundaries_pos, True, LOG)

if LOG:
    plt.loglog(abs(xdata_fit_neg),ydata_fit_neg, 'k--', alpha = 0.9 ,label = 'Fit -')
else:
    plt.plot(xdata_fit_neg,ydata_fit_neg, 'k--', alpha = 0.9 ,label = 'Fit -')

plt.legend()
plt.tight_layout()

if show:
    plt.show()
else:
    if LOG:
        plt.savefig(fig_name + '_log' + fig_extension)
    else:
        plt.savefig(fig_name + fig_extension)

## 3. Fitting Dynamical Critical Exponent

In [ ]:
# Imports chi data, fits to obtain z, plots fit including 3 data sets

import numpy as np
import matplotlib.pyplot as plt
from data_processing import f_z, fit_funct_z

# Specify where to find data sets 
data_dir = './exported_data/'

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'triple_fit_'
fig_name = fig_dir + identifier

# Data specifications
data_names = ['chi'] # Quantity
L_range = [10, 15, 20, 25, 30, 35, 40] # Data sets
T_data = 81 # Temperature steps 

# Initialization 
y_data = np.zeros((len(L_range), T_data))
y_err = np.zeros((len(L_range), T_data))
x_data = np.zeros((len(L_range), T_data))
x_err = np.zeros((len(L_range), T_data))
top_xy = np.zeros((len(L_range), 2))
top_xy_err = np.zeros((len(L_range), 2))

# Plot markup 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

for data_name in data_names:
    for i, L_i in enumerate(L_range):
        # Import data sets 
        
        name = 'saved_data_L' + str(L_i) +'_SW_2000.npz'
        npzfile = np.load(data_dir + name)
        y_data[i] = npzfile[data_name][:,0]
        y_err[i] = npzfile[data_name][:,1]
        
        j = np.argmax(y_data[i])    # Identify maximum of the dataset
        top_xy [i, 1]= y_data[i, j] # Sore values of the maximum 
        top_xy_err[i,1] = y_err[i,j]
        
        x_data[i] = npzfile['temperature'][:,0]
        top_xy[i, 0] = x_data[i,j]
        
        top_xy_err[i,0] = abs(npzfile['temperature'][j + 1,0] - npzfile['temperature'][j,0])/2
        
ii = 0
markers = ['+', 'x', '.']
for k, L_i in enumerate(L_range):
    if L_i in [10, 20, 40]:
        # Plot data for 3 data sets
        plt.errorbar(x_data[k], y_data[k], yerr=y_err[k], fmt=markers[ii%3], label = 'L = %.f'%L_i , markersize = 6, capsize = 4)
        ii += 1
plt.xlabel('$\mathrm{k_B T/J}$', fontsize=18)
plt.ylabel('$\chi$', fontsize=18)

# Fit to the data 
popt, fit_err = fit_funct_z(f_z, L_range, top_xy[:,0], top_xy_err[:,0], (-np.inf, np.inf))
popt2, fit_err2 = fit_funct_z(f_z, L_range, top_xy[:,1], top_xy_err[:,1], (-np.inf, np.inf))

# Calculation using the fitting results 
L_size = np.linspace(10, 40, 100)
x = f_z(L_size, *popt)
x_max = f_z(L_size, *popt + fit_err)
x_min = f_z(L_size, *popt - fit_err) 

y = f_z(L_size, *popt2)
y_max = f_z(L_size, *popt2 + fit_err2)
y_min = f_z(L_size, *popt2 - fit_err2)

# Plotting 
plt.plot(x, y, 'b-', label = 'Fit')
plt.legend()
plt.plot(x_max, y_max, 'b--')
plt.plot(x_min, y_min, 'b--')
plt.tight_layout()
print('z =', np.round(-popt[0], 2), '(', int(np.round(100*fit_err[0],0)), ')' )
plt.savefig(fig_name + data_name + '.png')
print('Figures are saved to:', fig_dir)
plt.close()

## 4. Simulation performance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from data_processing import f_sim, fit_funct_sim

# Speficy where and how to store figures 
fig_dir = './exported_figs/'
identifier = 'sim_performance'
fig_name = fig_dir + identifier

# Simulation data from general runs, all for the same ammount of MC steps.
time_sf = np.array([5.00, 20.66, 43.34 ,81.8])
L_sf = np.array([10, 20, 30, 40])
time_sw = np.array([3.1, 11.3, 25.8, 44.23])
L_sw = L_sf

norm = max(time_sf)
time_sf = time_sf/norm
time_sw = time_sw/norm

# Plot markup 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)

plt.plot(L_sf, time_sf, 'b+', markersize = 6, label = 'Metropolis')
plt.plot(L_sw, time_sw, 'x', color = 'orange', markersize = 6, label = 'Swendsen-Wang')
plt.legend()
# Fitting 
popt, fit_err = fit_funct_sim(f_sim, L_sf, time_sf)
popt2, fit_err2 = fit_funct_sim(f_sim, L_sw, time_sw)

L_range = np.linspace(10, 40, 100)
plt.plot(L_range, f_sim(L_range, *popt), 'b--', label = 'Metropolis fit')
plt.plot(L_range, f_sim(L_range, *popt2), '-.', color = 'orange', label = 'Swendsen-Wang fit')
plt.xlabel('L')
plt.ylabel('t')
plt.tight_layout()

print('Rough Metropolis behaviour:', np.round(popt[1], 2))
print('Rough Swendsen-Wang behaviour:', np.round(popt2[1], 2))

plt.savefig(fig_name + '.png')
print('Figure are saved to:', fig_dir)
plt.close()